# Comparison of Embedding Methods

### This notebook demonstrates the calculation of Adjusted Rand Scores on a variety of directed graphs. We will compare the performance of three spectral embedding methods: Adjacency Spectral Embedding (ASE), Laplacian Spectral Embedding (LSE), and Scikit-Learn's Spectral Embedding.

In [36]:
import time
import warnings
warnings.filterwarnings("ignore")
import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import adjusted_rand_score
from sklearn.manifold import spectral_embedding
from sklearn.mixture import GaussianMixture

from graspologic.embed import AdjacencySpectralEmbed, LaplacianSpectralEmbed

### The following function takes a k-neighbors graph sampled from two multivariate gaussian distributions. It then applies a specified spectral embedding method, followed by Gaussian Mixture Clustering to compute the Adjusted Rand Score. This process is repeated over multiple iterations, with each score being recorded.

In [37]:
def calc_ari(kn_graph, n_sims, embed_method, labels_true, cov_type):
    rows = []

    for _ in range(n_sims):

        #choose embedding method
        if embed_method == "ase":
            ase = AdjacencySpectralEmbed(n_components=2)
            Xhat, Yhat = ase.fit_transform(kn_graph)
        elif embed_method == "lse":
            lse = LaplacianSpectralEmbed(n_components=2)
            Xhat, Yhat = lse.fit_transform(kn_graph)
        elif embed_method == "sklearn":
            Xhat = spectral_embedding(kn_graph, n_components=2)

        #concatenate Xhat and Yhat if using ase or lse
        if embed_method == "ase" or embed_method == "lse":
            Xhat = np.concatenate((Xhat, Yhat), axis=1)

        #calculate the ARI score
        gm_ic = GaussianMixture(n_components=2, covariance_type=cov_type)
        labels_mclust = gm_ic.fit_predict(Xhat)
        ari = adjusted_rand_score(labels_true, labels_mclust)

        #save the embedding method used with the corresponding ARI value
        result = {
            "test": embed_method,
            "ari": ari
        }
        rows.append(result)


    results = pd.DataFrame(rows)
    return results

### Now we present 2 cases, in which LSE performs the best in case 1, while ASE performs the best in case 2. In each case, we choose a certain number of vertices, along with 2 vectors to represent the means of the multivariate distributions, and run the above method using all three embedding methods. We then calculate the means of the ARI scores for each embedding method across the number of iterations to determine which embedding method gave the best results.

In [40]:
#case 1: LSE does best
n_verts_lse = 250
n_sims = 1000
B_core = np.array([[0.011, 0.027], [0.027, 0.079]])
labels_lse = int(n_verts_lse/2) * [0] + int(n_verts_lse/2) * [1]

#generate kn graph

#make latent position matrix
mean_1_lse = np.array([-0.33, -0.94])
mean_2_lse = np.array([-0.94, 0.33])

#generate two gaussian distributions from the svd (cov=identity)
X = np.random.multivariate_normal(mean = mean_1_lse, cov = np.eye(2), size = int(n_verts_lse/2))
Y = np.random.multivariate_normal(mean = mean_2_lse, cov = np.eye(2), size = int(n_verts_lse/2))

#concatenate the two matrices to get the full latent position matrix
lat_mat = np.concatenate((X, Y), axis=0)

#get k_neighbors graph from latent position matrix (k=sqrt(n))
kn_graph = kneighbors_graph(lat_mat, n_neighbors=int(np.sqrt(n_verts_lse)))
kn_graph = kn_graph.toarray()

ari_aff_lse_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "lse", labels_true = labels_lse, cov_type = "full")
ari_aff_ase_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "ase", labels_true = labels_lse, cov_type = "full")
ari_aff_sklearn_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "sklearn", labels_true = labels_lse, cov_type = "full")
ari_aff_df = pd.concat([ari_aff_lse_df, ari_aff_ase_df, ari_aff_sklearn_df])

#groupby the means for each embedding method across the simulations
ari_aff_means = ari_aff_df.groupby(["test"]).mean().round(2)
ari_aff_means

,ari
test,
ase,0.07
lse,0.32
sklearn,0.21


In [39]:
#case 2: ASE does best
n_verts_ase = 800
n_sims = 1000
B_affinity = np.array([[0.050, 0.013], [0.013, 0.051]])
labels_ase = int(n_verts_ase/2) * [0] + int(n_verts_ase/2) * [1]

#generate kn graph

#means of our mvn distributions
mean_1_ase = np.array([-0.69, -0.72])
mean_2_ase = np.array([-0.72, 0.69])

#generate two gaussian distributions from the svd (cov=identity)
X = np.random.multivariate_normal(mean = mean_1_ase, cov = np.eye(2), size = int(n_verts_ase/2))
Y = np.random.multivariate_normal(mean = mean_2_ase, cov = np.eye(2), size = int(n_verts_ase/2))

#concatenate the two matrices to get the full latent position matrix
lat_mat = np.concatenate((X, Y), axis=0)

#get k_neighbors graph from latent position matrix (k=sqrt(n))
kn_graph = kneighbors_graph(lat_mat, n_neighbors=int(np.sqrt(n_verts_ase)))
kn_graph = kn_graph.toarray()

ari_core_ase_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "ase", labels_true = labels_ase, cov_type = "full")
ari_core_lse_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "lse", labels_true = labels_ase, cov_type = "full")
ari_core_sklearn_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "sklearn", labels_true = labels_ase, cov_type = "full")
ari_core_df = pd.concat([ari_core_ase_df, ari_core_lse_df, ari_core_sklearn_df])
ari_core_means = ari_core_df.groupby(["test"]).mean().round(2)
ari_core_means

,ari
test,
ase,0.20
lse,0.17
sklearn,0.13
